<a href="https://colab.research.google.com/github/mehdihemmatyar/RecommendationSystem/blob/main/MovieRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import zipfile

In [ ]:

# Specify the path to the ZIP file
zip_file_path = "/content/drive/MyDrive/dataset-ml-25m.zip"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    # Extract all files and directories in the ZIP file
    zip_ref.extractall("/content/drive/MyDrive")

In [2]:
genom_scores = pd.read_csv("/content/drive/MyDrive/ml-25m/genome-scores.csv")

In [3]:
genom_scores.head(10)

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075
5,1,6,0.14675
6,1,7,0.06350
7,1,8,0.20375
8,1,9,0.20200
9,1,10,0.03075


In [4]:
filtered_genom_scores = genom_scores[genom_scores['relevance']>0.8]

In [5]:
genom_tags = pd.read_csv("/content/drive/MyDrive/ml-25m/genome-tags.csv")

In [6]:
genom_tags

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [8]:
merged_genom_tag = filtered_genom_scores.merge(genom_tags, on='tagId')


In [9]:
merged_genom_tag.head()

,movieId,tagId,relevance,tag
0,1,29,0.89375,adventure
1,2,29,0.97600,adventure
2,10,29,0.80175,adventure
3,15,29,0.97475,adventure
4,146,29,0.83600,adventure


In [41]:
gp_genom_tag = merged_genom_tag.groupby('movieId')['tag'].apply(str)

In [30]:
gp_genom_tag

movieId
1         0                     adventure\n369          ...
2         1              adventure\n1621           child...
3         12827         comedy\n14116    good sequel\n14...
4         15008    chick flick\n15308        divorce\n15...
5         3189                           family\n12828  ...
                                ...                        
205072    4115         friendship\n9422           origin...
205076    34469    girlie movie\nName: 205076, dtype: ob...
205383        9423    original\nName: 205383, dtype: object
205425    9424             original\n14115              ...
206499    9425      original\n132749       pg-13\nName: ...
Name: tag, Length: 13498, dtype: object

In [12]:
movies = pd.read_csv("/content/drive/MyDrive/ml-25m/movies.csv")

In [13]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
ratings = pd.read_csv("/content/drive/MyDrive/ml-25m/ratings.csv") # we don't need this for content base because user rate's movies

In [16]:
tags = pd.read_csv("/content/drive/MyDrive/ml-25m/tags.csv")

In [17]:
# Remove duplicates
movies.drop_duplicates(inplace=True)
ratings.drop_duplicates(inplace=True)
tags.drop_duplicates(inplace=True)
genom_scores.drop_duplicates(inplace=True)
genom_tags.drop_duplicates(inplace=True)

# Fill in missing values, if any
movies.fillna('', inplace=True)
ratings.fillna(0, inplace=True)
tags.fillna('', inplace=True)
genom_scores.fillna(0, inplace=True)
genom_tags.fillna('', inplace=True)


In [51]:
movies['genres']

0        Adventure|Animation|Children|Comedy|Fantasy
1                         Adventure|Children|Fantasy
2                                     Comedy|Romance
3                               Comedy|Drama|Romance
4                                             Comedy
                            ...                     
62418                                          Drama
62419                                    Documentary
62420                                   Comedy|Drama
62421                             (no genres listed)
62422                         Action|Adventure|Drama
Name: genres, Length: 62423, dtype: object

In [61]:
movies['genres'] = movies['genres'].str.replace('-', " ")

In [62]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [65]:

#combine genres and tags and user's tage into a consolidated dataset
movie_genres = movies['genres'].str.replace('|', " ")
movie_tags = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

consolidated_data = pd.merge(movies, movie_tags, on='movieId')
consolidated_data = pd.merge(consolidated_data, gp_genom_tag, on='movieId')


<ipython-input-65-e36eb0026712>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_genres = movies['genres'].str.replace('|', " ")


In [66]:
consolidated_data

,movieId,title,genres,tag_x,tag_y
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,Owned imdb top 250 Pixar Pixar time travel chi...,0 adventure\n369 ...
1,2,Jumanji (1995),Adventure Children Fantasy,Robin Williams time travel fantasy based on ch...,1 adventure\n1621 child...
2,3,Grumpier Old Men (1995),Comedy Romance,funny best friend duringcreditsstinger fishing...,12827 comedy\n14116 good sequel\n14...
3,4,Waiting to Exhale (1995),Comedy Drama Romance,based on novel or book chick flick divorce int...,15008 chick flick\n15308 divorce\n15...
4,5,Father of the Bride Part II (1995),Comedy,aging baby confidence contraception daughter g...,3189 family\n12828 ...
...,...,...,...,...,...
13332,205072,Zombieland: Double Tap (2019),Action Comedy Horror,All-Star Cast Jesse Eisenberg Logo Joke The St...,4115 friendship\n9422 origin...
13333,205076,Downton Abbey (2019),Drama,period drama period drama theater,"34469 girlie movie\nName: 205076, dtype: ob..."
13334,205383,El Camino: A Breaking Bad Movie (2019),Crime Drama Thriller,breaking bad breaking bad cinematography Vince...,"9423 original\nName: 205383, dtype: object"
13335,205425,Dave Chappelle: Sticks & Stones (2019),Comedy,stand-up comedy stand-up comedy comedy Politic...,9424 original\n14115 ...


In [21]:
consolidated_data.shape

(13337, 25)

In [34]:
# Subset of movies for similarity calculation (faster process)
subset_movies = consolidated_data.head(5000)

In [35]:
subset_movies.head(3)

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,tag_x,tag_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,Owned imdb top 250 Pixar Pixar time travel chi...,0 adventure\n369 ...
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,Robin Williams time travel fantasy based on ch...,1 adventure\n1621 child...
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,funny best friend duringcreditsstinger fishing...,12827 comedy\n14116 good sequel\n14...


In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [27]:
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')


In [36]:
subset_genres_matrix = tfidf_vectorizer.fit_transform(subset_movies['genres']+subset_movies['tag_x']+subset_movies['tag_y'])


In [37]:
cosine_similarities = cosine_similarity(subset_genres_matrix, subset_genres_matrix)


In [38]:

# Function to get movie recommendations based on title similarity
def get_movie_recommendations(title, cosine_similarities, consolidated_data, top_n=5):
    # Get the index of the movie
    index = consolidated_data[consolidated_data['title'] == title].index[0]

    # similarities of the movie with other movies
    similarities = list(enumerate(cosine_similarities[index]))

    # Sort the movies based on similarity score
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Get the top-N similar movies (excluding the input movie itself)
    top_similar_movies = similarities[1:top_n + 1]

    # Get the movie titles for the recommended movies
    recommended_movies = consolidated_data.iloc[[movie[0] for movie in top_similar_movies]]['title'].tolist()
    return recommended_movies


In [40]:

# Example 
input_movie = 'Ice Age (2002)'
recommended_movies = get_movie_recommendations(input_movie, cosine_similarities, consolidated_data)
print(f"Recommended movies for '{input_movie}':")
for movie in recommended_movies:
    print(movie)

Recommended movies for 'Ice Age (2002)':
Bug's Life, A (1998)
Toy Story (1995)
Monsters, Inc. (2001)
Toy Story 2 (1999)
Antz (1998)
